In [1]:
from torch.nn import functional as F
import torch
from torch import nn

In [3]:
class RNN(nn.Module):
    
    def __init__(self,input_size,hidden_size,output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size,hidden_size,batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)
    
    
    def forward(self,x,target):
        # B,T,C
        out,h = self.rnn(x)
        # B,T,H
        out = self.fc(out)
        loss = F.binary_cross_entropy(out,target)
        return loss

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Initialize the hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)

        # Pass the input through the RNN layer
        out, hn = self.rnn(x, h0)

        # Pass the output through the fully connected layer and the sigmoid activation function
        out = self.fc(out)
        
        out = F.softmax(out,dim=-1)

        return out.squeeze()

# Generate some random binary classification data
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# Set up the RNN model
input_size = 10
hidden_size = 16
output_size = 1
rnn = RNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.AdamW(rnn.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    X = X_train.unsqueeze(0)
    Y = y_train.unsqueeze(0)
    # Forward pass
    outputs = rnn(X)

    # Compute the loss and backward pass
    Y = Y.squeeze(0)
    loss = criterion(outputs, Y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print('Epoch %d Loss: %.4f' % (epoch, loss.item()))

# Test the model
with torch.no_grad():
    y_pred = rnn(X_test)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0

accuracy = (y_pred == y_test).sum().item() / y_test.size(0)
print('Test Accuracy: %.4f' % accuracy)

Epoch 0 Loss: 49.6250
Epoch 10 Loss: 49.6250
Epoch 20 Loss: 49.6250
Epoch 30 Loss: 49.6250
Epoch 40 Loss: 49.6250
Epoch 50 Loss: 49.6250
Epoch 60 Loss: 49.6250
Epoch 70 Loss: 49.6250
Epoch 80 Loss: 49.6250
Epoch 90 Loss: 49.6250


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor